# XX
 - **Project:** GP2 Parkinson's disease meta-GWAS on European, Ashkenazi Jewish, Finnish, and Icelandic individuals 
 - **Versions:** R
 - **Last Updated:** January 2025

### Notebook Overview
xx

### CHANGELOG
* 06-MAR-2025: Notebook cleaned 
* 10-FEB-2025: Notebook started

In [ ]:
## Import the necessary packages 
import os
import numpy as np
import pandas as pd
import math
import numbers
import sys
import subprocess
import statsmodels.api as sm
import scipy
from scipy import stats
from scipy.stats import norm
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
from tqdm import tqdm

## Print out package versions
## Getting packages loaded into this notebook and their versions to allow for reproducibility
    # Repurposed code from stackoverflow here: https://stackoverflow.com/questions/40428931/package-for-listing-version-of-packages-used-in-a-jupyter-notebook

## Import packages 
import pkg_resources
import types
from datetime import date
today = date.today()
date = today.strftime("%d-%b-%Y").upper()

## Define function 
def get_imports():
    for name, val in globals().items():
        if isinstance(val, types.ModuleType):
            # Split ensures you get root package, not just imported function
            name = val.__name__.split(".")[0]

        elif isinstance(val, type):
            name = val.__module__.split(".")[0]

        # Some packages are weird and have different imported names vs. system/pip names
        # Unfortunately, there is no systematic way to get pip names from a package's imported name. You'll have to add exceptions to this list manually!
        poorly_named_packages = {
            "PIL": "Pillow",
            "sklearn": "scikit-learn"
        }
        if name in poorly_named_packages.keys():
            name = poorly_named_packages[name]

        yield name

## Get a list of packages imported 
imports = list(set(get_imports()))

# The only way I found to get the version of the root package from only the name of the package is to cross-check the names of installed packages vs. imported packages
requirements = []
for m in pkg_resources.working_set:
    if m.project_name in imports and m.project_name!="pip":
        requirements.append((m.project_name, m.version))

## Print out packages and versions 
print(f"PACKAGE VERSIONS ({date})")
for r in requirements:
    print("\t{}=={}".format(*r))

# Calculate ORs and CIs

In [ ]:
def calculate_ors_cis(input_file_path, work_dir, output_dir):
    with open(input_file_path, 'r') as file:
        file_paths = [line.strip().format(WORK_DIR=work_dir) for line in file.readlines()]

    os.makedirs(output_dir, exist_ok=True)

    for file_path in file_paths:
        data = pd.read_csv(file_path, sep='\t')
        
        if 'BETA' in data.columns and 'SE' in data.columns:
            # Calculate the odds ratio and the confidence intervals
            data['OR'] = np.exp(data['BETA'])
            data['L95'] = np.exp(data['BETA'] - 1.96 * data['SE'])
            data['U95'] = np.exp(data['BETA'] + 1.96 * data['SE'])
            
            base_name = os.path.basename(file_path)
            output_file_path = os.path.join(output_dir, base_name.replace('.txt', '.ORs.txt'))
            
            print(f"Processed: {output_file_path}")
            data.to_csv(output_file_path, sep='\t', index=False)
        else:
            print(f"Warning: 'BETA' and 'SE' columns not found in {file_path}")

In [ ]:
## Run through files 
calculate_ors_cis(f'{FP_DIR}/data/paths_21cohorts.txt', f'{WORK_DIR}', f'{FP_DIR}/data/processed/')

## Get the summary from the final file 

In [ ]:
def calculate_ors_cis_meta(input_file_path, output_file_path):
    # Load the data from the input file
    data = pd.read_csv(input_file_path, sep='\t')
    
    # Check if the required columns are present
    if 'Effect' in data.columns and 'StdErr' in data.columns:
        # Calculate the odds ratio and the confidence intervals
        data['OR'] = np.exp(data['Effect'])
        data['L95'] = np.exp(data['Effect'] - 1.96 * data['StdErr'])
        data['U95'] = np.exp(data['Effect'] + 1.96 * data['StdErr'])
        
        # Save the updated dataframe to the output file
        os.makedirs(os.path.dirname(output_file_path), exist_ok=True)
        data.to_csv(output_file_path, sep='\t', index=False)
        
        print(f"Processed: {output_file_path}")
    else:
        print(f"Warning: 'Effect' and 'StdErr' columns not found in {input_file_path}")

In [ ]:
## Run through files 
calculate_ors_cis_meta(f'{WORK_DIR}/METAS/METAANALYSIS1.TBL', f'{FP_DIR}/data/processed/METAANALYSIS1.TBL.ORs.txt')

## Extract rsID 

In [ ]:
! head {FP_DIR}/data/processed/deCODE_CC_FOR_PLINK_MAF.ORs.txt

SNP	CHR	BP	A1	A2	BETA	SE	P	NMISS	OR	L95	U95
chr1:596477:G:A	1	596477	A	G	-0.0080321701243519	0.094380997121334	0.93211	18429.84873202498	0.9920000015603291	0.8244663713923294	1.19357688468705
chr1:598745:C:CAG	1	598745	CAG	C	0.0079681696370244	0.0569300018250942	0.8827	18429.84873202498	1.0079999999877503	0.9015726659908295	1.1269906889409174
chr1:633423:C:T	1	633423	T	C	0.1638180017471313	0.0564690008759498	0.0038085	18429.84873202498	1.1779999016578975	1.054575967960476	1.315868946823966
chr1:701203:G:T	1	701203	T	G	-0.2344570010900497	0.1253699958324432	0.061086	18429.84873202498	0.7910002453084649	0.6186706641215313	1.011331915933779
chr1:702040:C:T	1	702040	T	C	0.027615200728178	0.0546800009906291	0.61888	18429.84873202498	1.0280000346386708	0.923524822286076	1.1442941713262944
chr1:710225:T:A	1	710225	A	T	-0.0020020001102238	0.042833000421524	0.9603	18429.84873202498	0.9980000025553284	0.9176358166737645	1.08540227724626
chr1:727233:G:A	1	727233	A	G	0.0198026001453399	0.054010000

In [ ]:
df = pd.read_csv(f'{FP_DIR}/data/processed/METAANALYSIS1.TBL.ORs.txt', sep="\t")

df = df.rename(columns={
    'MarkerName': 'SNP',
    'Allele1': 'A1',
    'Allele2': 'A2',
    'Effect': 'BETA',
    'StdErr': 'SE',
    'P-value': 'P',
    'OR': 'OR',
    'L95': 'L95',
    'U95': 'U95'
})

df['CHR'] = df['SNP'].apply(lambda x: x.split(':')[0][3:])
df['BP'] = df['SNP'].apply(lambda x: int(x.split(':')[1]))

df['A1'] = df['A1'].str.upper()
df['A2'] = df['A2'].str.upper()
df['NMISS'] = 0 ## placeholder to keep structure 

df = df[['SNP', 'CHR', 'BP', 'A1', 'A2', 'BETA', 'SE', 'P', 'NMISS', 'OR', 'L95', 'U95']]

# Print the dataframe to check
print(df.head())

                  SNP CHR         BP A1 A2    BETA      SE       P  NMISS  \
0  chr11:12541586:A:G  11   12541586  A  G -0.0016  0.0074  0.8270      0   
1   chr4:70553471:A:G   4   70553471  A  G  0.0095  0.0083  0.2555      0   
2  chr13:55101557:T:C  13   55101557  T  C  0.0116  0.0384  0.7621      0   
3  chr2:223892881:C:T   2  223892881  T  C -0.1172  0.2445  0.6319      0   
4   chr3:97729697:A:G   3   97729697  A  G  0.0646  0.0397  0.1040      0   

         OR       L95       U95  
0  0.998401  0.984025  1.012988  
1  1.009545  0.993255  1.026103  
2  1.011668  0.938320  1.090748  
3  0.889407  0.550780  1.436228  
4  1.066732  0.986875  1.153052  


In [ ]:
df.to_csv(f'{FP_DIR}/data/processed/METAANALYSIS.processed.ORs.txt', index=False, sep="\t")

In [ ]:
import os
import pandas as pd
from tqdm import tqdm

# Define the file paths
rsids_file = f"{FP_DIR}/data/rsIDs_hg38_variantIDs_cojo.txt"
input_file = f"{FP_DIR}/data/rsIDs_across_cohorts.txt"
cohort_names_file = f"{FP_DIR}/data/cohort_names.txt"

output_file = f"{FP_DIR}/data/rsIDs_cojo_with_ORs_allCohorts.txt"
split_output_dir = f"{FP_DIR}/data/rsID_files_withMeta_cojo/"

# Create output directory for split files if it doesn't exist
os.makedirs(split_output_dir, exist_ok=True)

# Function to clean and standardize filenames
def clean_filename(filename):
    return filename.strip().replace(" ", "_").replace("'", "")

# Read in data files
rsids_df = pd.read_csv(rsids_file, sep="\t")
input_df = pd.read_csv(input_file, sep="\t")
cohort_map_df = pd.read_csv(cohort_names_file, sep="\t")

# Create a mapping dictionary for Filename -> Rename
filename_to_cohort_map = dict(zip(cohort_map_df['Filename'], cohort_map_df['Rename']))

# Filter the input DataFrame to keep only rows with SNPs that are in the snp_to_rsid_map
filtered_df = input_df[input_df['SNP'].isin(dict(zip(rsids_df['SNP'], rsids_df['rsID'])).keys())].copy()
filtered_df['rsID'] = filtered_df['SNP'].map(dict(zip(rsids_df['SNP'], rsids_df['rsID'])))

# Map the old filenames to new cohort names
filtered_df['Cohort'] = filtered_df['Filename'].map(filename_to_cohort_map)

# Select the desired columns and reorder them
desired_columns = ['rsID', 'SNP', 'Cohort', 'BETA', 'SE', 'P', 'OR', 'L95', 'U95']
reordered_df = filtered_df[desired_columns]
reordered_df.to_csv(output_file, sep="\t", index=False)

# Loop through each unique rsID and save separate files for each
for rsid, group_df in tqdm(reordered_df.groupby('rsID'), desc="Saving individual rsID files", total=reordered_df['rsID'].nunique()):
    cleaned_rsid = clean_filename(rsid)  # Clean the rsID for consistent filenames
    output_split_file = os.path.join(split_output_dir, f"{cleaned_rsid}_split.txt")
    group_df.to_csv(output_split_file, sep="\t", index=False)

# Final messages
print(f"Processing complete. Filtered and reordered file saved as: {output_file}")
print(f"Individual rsID files saved in: {split_output_dir}")

In [ ]:
! ls {FP_DIR}/data/rsID_files_withMeta_cojo | wc -l

155


In [ ]:
%%bash

echo "./GP2/POP_CONTROLS/GP2_EUR_CC_FOR_PLINK_MAF_AGE_POP_CONTROLS.txt"
head -1 ./GP2/POP_CONTROLS/GP2_EUR_CC_FOR_PLINK_MAF_AGE_POP_CONTROLS.txt
grep "chr12:40340400:G:A" ./GP2/POP_CONTROLS/GP2_EUR_CC_FOR_PLINK_MAF_AGE_POP_CONTROLS.txt
echo ""

echo "./IPDGC/IPDGC_COURAGE_UK_EUR_CC_FOR_PLINK_MAF.txt"
head -1 ./IPDGC/IPDGC_COURAGE_UK_EUR_CC_FOR_PLINK_MAF.txt
grep "chr12:40340400:G:A" ./IPDGC/IPDGC_COURAGE_UK_EUR_CC_FOR_PLINK_MAF.txt
echo ""

echo "./IPDGC/IPDGC_DUTCH_EUR_CC_FOR_PLINK_MAF.txt"
head -1 ./IPDGC/IPDGC_DUTCH_EUR_CC_FOR_PLINK_MAF.txt
grep "chr12:40340400:G:A" ./IPDGC/IPDGC_DUTCH_EUR_CC_FOR_PLINK_MAF.txt
echo ""

echo "./IPDGC/IPDGC_GERMANY_EUR_CC_FOR_PLINK_MAF.txt"
head -1 ./IPDGC/IPDGC_GERMANY_EUR_CC_FOR_PLINK_MAF.txt
grep "chr12:40340400:G:A" ./IPDGC/IPDGC_GERMANY_EUR_CC_FOR_PLINK_MAF.txt
echo ""

echo "./IPDGC/IPDGC_MCGILL_EUR_CC_FOR_PLINK_MAF.txt"
head -1 ./IPDGC/IPDGC_MCGILL_EUR_CC_FOR_PLINK_MAF.txt
grep "chr12:40340400:G:A" ./IPDGC/IPDGC_MCGILL_EUR_CC_FOR_PLINK_MAF.txt
echo ""

echo "./IPDGC/IPDGC_MF_EUR_CC_FOR_PLINK_MAF.txt"
head -1 ./IPDGC/IPDGC_MF_EUR_CC_FOR_PLINK_MAF.txt
grep "chr12:40340400:G:A" ./IPDGC/IPDGC_MF_EUR_CC_FOR_PLINK_MAF.txt
echo ""

echo "./IPDGC/IPDGC_NIA_EUR_CC_FOR_PLINK_MAF.txt"
head -1 ./IPDGC/IPDGC_NIA_EUR_CC_FOR_PLINK_MAF.txt
grep "chr12:40340400:G:A" ./IPDGC/IPDGC_NIA_EUR_CC_FOR_PLINK_MAF.txt
echo ""

echo "./IPDGC/IPDGC_OSLO_EUR_CC_FOR_PLINK_PD.txt"
head -1 ./IPDGC/IPDGC_OSLO_EUR_CC_FOR_PLINK_PD.txt
grep "chr12:40340400:G:A" ./IPDGC/IPDGC_OSLO_EUR_CC_FOR_PLINK_PD.txt
echo ""

echo "./IPDGC/IPDGC_SHULMAN_EUR_CC_FOR_PLINK_MAF.txt"
head -1 ./IPDGC/IPDGC_SHULMAN_EUR_CC_FOR_PLINK_MAF.txt
grep "chr12:40340400:G:A" ./IPDGC/IPDGC_SHULMAN_EUR_CC_FOR_PLINK_MAF.txt
echo ""

echo "./IPDGC/IPDGC_SPAIN3_EUR_CC_NO_AGE_COVAR_FOR_PLINK_MAF.txt"
head -1 ./IPDGC/IPDGC_SPAIN3_EUR_CC_NO_AGE_COVAR_FOR_PLINK_MAF.txt
grep "chr12:40340400:G:A" ./IPDGC/IPDGC_SPAIN3_EUR_CC_NO_AGE_COVAR_FOR_PLINK_MAF.txt
echo ""

echo "./IPDGC/IPDGC_TUBI_EUR_CC_FOR_PLINK_MAF.txt"
head -1 ./IPDGC/IPDGC_TUBI_EUR_CC_FOR_PLINK_MAF.txt
grep "chr12:40340400:G:A" ./IPDGC/IPDGC_TUBI_EUR_CC_FOR_PLINK_MAF.txt
echo ""

echo "./IPDGC/IPDGC_UK_GWAS_EUR_CC_FOR_PLINK_MAF.txt"
head -1 ./IPDGC/IPDGC_UK_GWAS_EUR_CC_FOR_PLINK_MAF.txt
grep "chr12:40340400:G:A" ./IPDGC/IPDGC_UK_GWAS_EUR_CC_FOR_PLINK_MAF.txt
echo ""

echo "./METAS/FIGS_META_EUR_MAF_FOR_PLINK.txt"
head -1 ./METAS/FIGS_META_EUR_MAF_FOR_PLINK.txt
grep "chr12:40340400:G:A" ./METAS/FIGS_META_EUR_MAF_FOR_PLINK.txt
echo ""

echo "./UKB/UKB_ALL/UKB_ALL_CC_FOR_PLINK_MAF"
head -1 ./UKB/UKB_ALL/UKB_ALL_CC_FOR_PLINK_MAF
grep "chr12:40340400:G:A" ./UKB/UKB_ALL/UKB_ALL_CC_FOR_PLINK_MAF
echo ""

echo "./GP2/UPDATED/GP2_AJ_CC_FOR_PLINK_MAF_update.txt"
head -1 ./GP2/UPDATED/GP2_AJ_CC_FOR_PLINK_MAF_update.txt
grep "chr12:40340400:G:A" ./GP2/UPDATED/GP2_AJ_CC_FOR_PLINK_MAF_update.txt
echo ""

echo "./FIGS/FIGS_AJ_CC_FOR_PLINK_MAF.txt"
head -1 ./FIGS/FIGS_AJ_CC_FOR_PLINK_MAF.txt
grep "chr12:40340400:G:A" ./FIGS/FIGS_AJ_CC_FOR_PLINK_MAF.txt
echo ""

echo "./FINNGEN_R10_PD/finngen_R11_G6_PARKINSON_FOR_PLINK"
head -1 ./FINNGEN_R10_PD/finngen_R11_G6_PARKINSON_FOR_PLINK
grep "chr12:40340400:G:A" ./FINNGEN_R10_PD/finngen_R11_G6_PARKINSON_FOR_PLINK
echo ""

echo "./IPDGC/IPDGC_FINLAND_FIN_CC_NO_AGE_FOR_PLINK.txt"
head -1 ./IPDGC/IPDGC_FINLAND_FIN_CC_NO_AGE_FOR_PLINK.txt
grep "chr12:40340400:G:A" ./IPDGC/IPDGC_FINLAND_FIN_CC_NO_AGE_FOR_PLINK.txt
echo ""

echo "./deCODE/deCODE_CC_FOR_PLINK_MAF.txt"
head -1 ./deCODE/deCODE_CC_FOR_PLINK_MAF.txt
grep "chr12:40340400:G:A" ./deCODE/deCODE_CC_FOR_PLINK_MAF.txt
echo ""

echo "./MVP_CC_FOR_PLINK_MAF.txt"
head -1 ./MVP_CC_FOR_PLINK_MAF.txt
grep "chr12:40340400:G:A" ./MVP_CC_FOR_PLINK_MAF.txt
echo ""

echo "./UKB/UKB_PROXY/UKB_PROXY_ADJUSTED_FOR_PLINK.txt"
head -1 ./UKB/UKB_PROXY/UKB_PROXY_ADJUSTED_FOR_PLINK.txt
grep "chr12:40340400:G:A" ./UKB/UKB_PROXY/UKB_PROXY_ADJUSTED_FOR_PLINK.txt
echo ""

In [ ]:
! head -1 {WORK_DIR}/METAS/EUR_ALL_WITH_PROXY_META_MAF_MVP.lrrk2_special_filter
! grep "chr12:40340400:G:A" {WORK_DIR}/METAS/EUR_ALL_WITH_PROXY_META_MAF_MVP.lrrk2_special_filter

CHR	BP	SNP	A1	A2	N	P	P(R)	BETA	BETA(R)	Q	I	WEIGHTED_Z	P(WZ)	F0	F1	F2	F3	F4	F5	F6	F7	F8	F9	F10	F11	F12	F13	F14	F15	F16	F17	F18	F19	F20
12	40340400	chr12:40340400:G:A	A	G	8	5.892e-89	5.343e-30	2.208	2.3254	0.0031	67.39			3.278									2.1774			3.0949	1.8799	2.0018	2.1167				1.6378	2.983


In [ ]:
g2019s = pd.read_csv(f'./../data/LRRK2-G2019S-rs34637584.tsv', sep="\t")
g2019s

,rsID,SNP(hg38),Cohort,BETA,P
0,rs34637584,chr12:40340400:G:A,[EUR] GP2,3.2780,3.440000e-10
1,rs34637584,chr12:40340400:G:A,[EUR] IPDGC - Spain,2.1774,6.490000e-09
2,rs34637584,chr12:40340400:G:A,[EUR] FIGS,3.0949,1.740000e-20
3,rs34637584,chr12:40340400:G:A,[EUR] UKBiobank - Case/Control,1.8799,3.460000e-07
4,rs34637584,chr12:40340400:G:A,[AJ] GP2,2.0018,9.180000e-09
5,rs34637584,chr12:40340400:G:A,[AJ] FIGS,2.1167,2.340000e-25
6,rs34637584,chr12:40340400:G:A,[EUR] Million Veteran Program,1.6378,1.540000e-11
7,rs34637584,chr12:40340400:G:A,[EUR] UKBiobank - Proxy Cases,2.9830,2.040000e-12


In [ ]:
g2019s["SE"] = np.abs(g2019s["BETA"] / norm.ppf(g2019s["P"] / 2))

g2019s["OR"] = np.exp(g2019s["BETA"])
g2019s["L95"] = np.exp(g2019s["BETA"] - 1.96 * g2019s["SE"])
g2019s["U95"] = np.exp(g2019s["BETA"] + 1.96 * g2019s["SE"])

display(g2019s)

,rsID,SNP(hg38),Cohort,BETA,P,SE,OR,L95,U95
0,rs34637584,chr12:40340400:G:A,[EUR] GP2,3.2780,3.440000e-10,0.522180,26.522674,9.530784,73.808438
1,rs34637584,chr12:40340400:G:A,[EUR] IPDGC - Spain,2.1774,6.490000e-09,0.375180,8.823336,4.229346,18.407398
2,rs34637584,chr12:40340400:G:A,[EUR] FIGS,3.0949,1.740000e-20,0.333603,22.085030,11.484957,42.468470
3,rs34637584,chr12:40340400:G:A,[EUR] UKBiobank - Case/Control,1.8799,3.460000e-07,0.368861,6.552850,3.180161,13.502411
4,rs34637584,chr12:40340400:G:A,[AJ] GP2,2.0018,9.180000e-09,0.348429,7.402368,3.739231,14.654099
5,rs34637584,chr12:40340400:G:A,[AJ] FIGS,2.1167,2.340000e-25,0.203421,8.303690,5.573341,12.371622
6,rs34637584,chr12:40340400:G:A,[EUR] Million Veteran Program,1.6378,1.540000e-11,0.242850,5.143841,3.195725,8.279529
7,rs34637584,chr12:40340400:G:A,[EUR] UKBiobank - Proxy Cases,2.9830,2.040000e-12,0.424220,19.746969,8.597986,45.352804


In [ ]:
g2019s.to_csv('./../data/LRRK2-G2019S-rs34637584_with_SE_OR_L95_U95.tsv', sep="\t", index=False)

In [ ]:
! head -1 {WORK_DIR}/METAS/EUR_ALL_WITH_PROXY_META_MAF_MVP_beta_fix.meta
# rs11929238
! grep "chr3:161328149:A:T" {WORK_DIR}/METAS/EUR_ALL_WITH_PROXY_META_MAF_MVP_beta_fix.meta

# rs1971534
! grep "chr16:52939375:A:G" {WORK_DIR}/METAS/EUR_ALL_WITH_PROXY_META_MAF_MVP_beta_fix.meta

# rs56384862
! grep "chr3:58410136:A:G" {WORK_DIR}/METAS/EUR_ALL_WITH_PROXY_META_MAF_MVP_beta_fix.meta

# rs72712862
! grep "chr1:161000299:C:T" {WORK_DIR}/METAS/EUR_ALL_WITH_PROXY_META_MAF_MVP_beta_fix.meta

# rs10569457
! grep "chr3:151990484:CGAAA:C" {WORK_DIR}/METAS/EUR_ALL_WITH_PROXY_META_MAF_MVP_beta_fix.meta

# rs41316625
! grep "chr6:29015989:G:A" {WORK_DIR}/METAS/EUR_ALL_WITH_PROXY_META_MAF_MVP_beta_fix.meta

 CHR          BP            SNP  A1  A2   N           P        P(R)    BETA BETA(R)       Q       I  WEIGHTED_Z       P(WZ)      F0      F1      F2      F3      F4      F5      F6      F7      F8      F9     F10     F11     F12     F13     F14     F15     F16     F17     F18     F19     F20
   3   161328149 chr3:161328149:A:T   T   A  20   7.365e-13   2.312e-07  0.0572  0.0657  0.0252   42.12       6.408   1.479e-10  0.0844  0.2715  0.2099  0.1067 -0.0275  0.1448  0.0916  0.0742  0.0827  0.0244  0.0608  0.1455  0.0419  0.0483  0.0932  0.0833  0.0065  0.1100  0.0658      NA  0.0127
  16    52939375 chr16:52939375:A:G   G   A  20   2.308e-09   1.975e-05  0.0537  0.0622  0.0221   42.96       5.676   1.382e-08  0.0944  0.0552  0.0745  0.2081 -0.0522  0.0552  0.1863  0.1134  0.1968  0.1190  0.1324  0.0412  0.0313  0.0474  0.0485  0.3130 -0.0114  0.1949  0.0141      NA  0.0433
   3    58410136 chr3:58410136:A:G   G   A  20   4.216e-08   4.216e-08  0.0429  0.0429  0.5971    0.00       5.472  

In [ ]:
file_path = f'{WORK_DIR}/METAS/FINAL_SETS/FOREST_PLOTS/data/error_3_rsIDs.txt'

# Reading the tab-delimited file
snp_data = pd.read_csv(file_path, delimiter='\t')

# Mapping old column names to new ones
column_renames = {
    "F0": "[EUR] GP2",
    "F1": "[EUR] IPDGC - Courage UK",
    "F2": "[EUR] IPDGC - Dutch",
    "F3": "[EUR] IPDGC - Germany",
    "F4": "[EUR] IPDGC - McGill",
    "F5": "[EUR] IPDGC - MF",
    "F6": "[EUR] IPDGC - NIA",
    "F7": "[EUR] IPDGC - Oslo",
    "F8": "[EUR] IPDGC - Shulman",
    "F9": "[EUR] IPDGC - Spain",
    "F10": "[EUR] IPDGC - Tübingen",
    "F11": "[EUR] IPDGC - UK GWAS",
    "F12": "[EUR] FIGS",
    "F13": "[EUR] UKBiobank - Case/Control",
    "F14": "[AJ] GP2",
    "F15": "[AJ] FIGS",
    "F16": "[FIN] FinnGen - R11",
    "F17": "[FIN] IPDGC - Finland",
    "F18": "[ICE] deCODE",
    "F19": "[EUR] Million Veteran Program",
    "F20": "[EUR] UKBiobank - Proxy Cases"
}

snp_data.rename(columns=column_renames, inplace=True)
display(snp_data.head())

,CHR,BP,SNP,A1,A2,N,P,P(R),BETA,BETA(R),...,[EUR] IPDGC - UK GWAS,[EUR] FIGS,[EUR] UKBiobank - Case/Control,[AJ] GP2,[AJ] FIGS,[FIN] FinnGen - R11,[FIN] IPDGC - Finland,[ICE] deCODE,[EUR] Million Veteran Program,[EUR] UKBiobank - Proxy Cases
0,3,161328149,chr3:161328149:A:T,T,A,20,7.365000e-13,2.312000e-07,0.0572,0.0657,...,0.1455,0.0419,0.0483,0.0932,0.0833,0.0065,0.1100,0.0658,NaN,0.0127
1,16,52939375,chr16:52939375:A:G,G,A,20,2.308000e-09,1.975000e-05,0.0537,0.0622,...,0.0412,0.0313,0.0474,0.0485,0.3130,-0.0114,0.1949,0.0141,NaN,0.0433
2,3,58410136,chr3:58410136:A:G,G,A,20,4.216000e-08,4.216000e-08,0.0429,0.0429,...,0.0069,0.0535,0.0324,0.0541,-0.1088,0.0228,0.0828,0.0611,NaN,0.0531
3,1,161000299,chr1:161000299:C:T,T,C,18,4.276000e-10,1.090000e-07,0.2103,0.2188,...,0.2000,0.2449,0.2392,0.5484,0.7396,0.1973,-0.1397,0.3716,NaN,0.1452
4,3,151990484,chr3:151990484:CGAAA:C,C,CGAAA,20,1.611000e-08,1.611000e-08,-0.0532,-0.0532,...,-0.0099,-0.0607,-0.0780,-0.0089,0.0232,-0.0498,-0.1185,-0.0576,NaN,-0.0371


In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import norm

# Dictionary mapping SNP identifiers to rsIDs
snp_to_rsID = {
    "chr3:161328149:A:T": "rs11929238",
    "chr16:52939375:A:G": "rs1971534",
    "chr3:58410136:A:G": "rs56384862",
    "chr1:161000299:C:T": "rs72712862",
    "chr3:151990484:CGAAA:C": "rs10569457",
    "chr6:29015989:G:A": "rs41316625"
}

# Define the file paths for each cohort
files_info = {
    "[EUR] GP2": "/data/CARD_AA/projects/2024_GP2_EUROPEAN_META/GP2/POP_CONTROLS/GP2_EUR_CC_FOR_PLINK_MAF_AGE_POP_CONTROLS.txt",
    "[EUR] IPDGC - Courage UK": "/data/CARD_AA/projects/2024_GP2_EUROPEAN_META/IPDGC/IPDGC_COURAGE_UK_EUR_CC_FOR_PLINK_MAF.txt",
    "[EUR] IPDGC - Dutch": "/data/CARD_AA/projects/2024_GP2_EUROPEAN_META/IPDGC/IPDGC_DUTCH_EUR_CC_FOR_PLINK_MAF.txt",
    "[EUR] IPDGC - Germany": "/data/CARD_AA/projects/2024_GP2_EUROPEAN_META/IPDGC/IPDGC_GERMANY_EUR_CC_FOR_PLINK_MAF.txt",
    "[EUR] IPDGC - McGill": "/data/CARD_AA/projects/2024_GP2_EUROPEAN_META/IPDGC/IPDGC_MCGILL_EUR_CC_FOR_PLINK_MAF.txt",
    "[EUR] IPDGC - MF": "/data/CARD_AA/projects/2024_GP2_EUROPEAN_META/IPDGC/IPDGC_MF_EUR_CC_FOR_PLINK_MAF.txt",
    "[EUR] IPDGC - NIA": "/data/CARD_AA/projects/2024_GP2_EUROPEAN_META/IPDGC/IPDGC_NIA_EUR_CC_FOR_PLINK_MAF.txt",
    "[EUR] IPDGC - Oslo": "/data/CARD_AA/projects/2024_GP2_EUROPEAN_META/IPDGC/IPDGC_OSLO_EUR_CC_FOR_PLINK_PD.txt",
    "[EUR] IPDGC - Shulman": "/data/CARD_AA/projects/2024_GP2_EUROPEAN_META/IPDGC/IPDGC_SHULMAN_EUR_CC_FOR_PLINK_MAF.txt",
    "[EUR] IPDGC - Spain": "/data/CARD_AA/projects/2024_GP2_EUROPEAN_META/IPDGC/IPDGC_SPAIN3_EUR_CC_NO_AGE_COVAR_FOR_PLINK_MAF.txt",
    "[EUR] IPDGC - Tübingen": "/data/CARD_AA/projects/2024_GP2_EUROPEAN_META/IPDGC/IPDGC_TUBI_EUR_CC_FOR_PLINK_MAF.txt",
    "[EUR] IPDGC - UK GWAS": "/data/CARD_AA/projects/2024_GP2_EUROPEAN_META/IPDGC/IPDGC_UK_GWAS_EUR_CC_FOR_PLINK_MAF.txt",
    "[EUR] FIGS": "/data/CARD_AA/projects/2024_GP2_EUROPEAN_META/METAS/FIGS_META_EUR_MAF_FOR_PLINK.txt",
    "[EUR] UKBiobank - Case/Control": "/data/CARD_AA/projects/2024_GP2_EUROPEAN_META/UKB/UKB_ALL/UKB_ALL_CC_FOR_PLINK_MAF",
    "[AJ] GP2": "/data/CARD_AA/projects/2024_GP2_EUROPEAN_META/GP2/UPDATED/GP2_AJ_CC_FOR_PLINK_MAF_update.txt",
    "[AJ] FIGS": "/data/CARD_AA/projects/2024_GP2_EUROPEAN_META/FIGS/FIGS_AJ_CC_FOR_PLINK_MAF.txt",
    "[FIN] FinnGen - R11": "/data/CARD_AA/projects/2024_GP2_EUROPEAN_META/FINNGEN_R10_PD/finngen_R11_G6_PARKINSON_FOR_PLINK",
    "[FIN] IPDGC - Finland": "/data/CARD_AA/projects/2024_GP2_EUROPEAN_META/IPDGC/IPDGC_FINLAND_FIN_CC_NO_AGE_FOR_PLINK.txt",
    "[ICE] deCODE": "/data/CARD_AA/projects/2024_GP2_EUROPEAN_META/deCODE/deCODE_CC_FOR_PLINK_MAF.txt",
    "[EUR] Million Veteran Program": "/data/CARD_AA/projects/2024_GP2_EUROPEAN_META/MVP_CC_FOR_PLINK_MAF.txt",
    "[EUR] UKBiobank - Proxy Cases": "/data/CARD_AA/projects/2024_GP2_EUROPEAN_META/UKB/UKB_PROXY/UKB_PROXY_ADJUSTED_FOR_PLINK.txt"
}

results = pd.DataFrame()

# Process each file
for cohort, file_path in files_info.items():
    # Load data from each file (assuming tab-delimited with header)
    df = pd.read_csv(file_path, delimiter='\t')
    
    # Filter for each SNP in snp_data and calculate statistics
    for _, snp in snp_data.iterrows():
        snp_of_interest = snp['SNP']
        filtered_data = df[df['SNP'] == snp_of_interest].copy()

        if not filtered_data.empty:
            # Calculate and set using loc to avoid SettingWithCopyWarning
            filtered_data.loc[:, 'SE'] = np.abs(filtered_data['BETA'] / norm.ppf(filtered_data['P'] / 2))
            filtered_data.loc[:, 'OR'] = np.exp(filtered_data['BETA'])
            filtered_data.loc[:, 'L95'] = np.exp(filtered_data['BETA'] - 1.96 * filtered_data['SE'])
            filtered_data.loc[:, 'U95'] = np.exp(filtered_data['BETA'] + 1.96 * filtered_data['SE'])

            # Append the necessary information to the results DataFrame
            for _, row in filtered_data.iterrows():
                rsID = snp_to_rsID.get(snp_of_interest, "Unknown")  # Default to "Unknown" if SNP not in dictionary
                results = results.append({
                    "rsID": rsID,
                    "SNP(hg38)": snp_of_interest,
                    "Cohort": cohort,
                    "BETA": row['BETA'],
                    "P": row['P'],
                    "SE": row['SE'],
                    "OR": row['OR'],
                    "L95": row['L95'],
                    "U95": row['U95']
                }, ignore_index=True)


results = results.sort_values(by='rsID')
display(results)


,rsID,SNP(hg38),Cohort,BETA,P,SE,OR,L95,U95
109,rs10569457,chr3:151990484:CGAAA:C,[ICE] deCODE,-0.057629,0.062097,0.030890,0.944000,0.888541,1.002920
27,rs10569457,chr3:151990484:CGAAA:C,[EUR] IPDGC - McGill,0.076878,0.482758,0.109532,1.079910,0.871268,1.338515
38,rs10569457,chr3:151990484:CGAAA:C,[EUR] IPDGC - NIA,-0.078313,0.336272,0.081444,0.924675,0.788248,1.084715
91,rs10569457,chr3:151990484:CGAAA:C,[AJ] FIGS,0.023228,0.780758,0.083455,1.023500,0.869060,1.205386
22,rs10569457,chr3:151990484:CGAAA:C,[EUR] IPDGC - Germany,-0.065666,0.536008,0.106107,0.936444,0.760609,1.152928
...,...,...,...,...,...,...,...,...,...
78,rs72712862,chr1:161000299:C:T,[EUR] UKBiobank - Case/Control,0.239174,0.016003,0.099290,1.270200,1.045573,1.543085
66,rs72712862,chr1:161000299:C:T,[EUR] IPDGC - UK GWAS,0.200022,0.308008,0.196214,1.221430,0.831473,1.794275
72,rs72712862,chr1:161000299:C:T,[EUR] FIGS,0.244900,0.049260,0.124546,1.277494,1.000789,1.630703
108,rs72712862,chr1:161000299:C:T,[ICE] deCODE,0.371564,0.000516,0.107008,1.450001,1.175658,1.788361


In [ ]:
results.to_csv(f'{WORK_DIR}/METAS/FINAL_SETS/FOREST_PLOTS/data/remaining_cojo_rsIDs_ORs.txt', index=False)

## Make one big file

In [ ]:
cojo_ors_file1 = pd.read_csv(f'{WORK_DIR}/METAS/FINAL_SETS/FOREST_PLOTS/data/rsIDs_cojo_with_ORs_allCohorts.txt', sep="\t")
cojo_ors_file2 = pd.read_csv(f'{WORK_DIR}/METAS/FINAL_SETS/FOREST_PLOTS/data/remaining_cojo_rsIDs_ORs.txt')

In [ ]:
cojo_ors_file1 = cojo_ors_file1[cojo_ors_file1['Cohort'] != '[TOTAL] Meta-analysis']
cojo_ors_file1

,rsID,SNP,Cohort,BETA,SE,P,OR,L95,U95
0,rs377808,chr1:52724687:T:G,[AJ] FIGS,0.050931,0.071870,0.478523,1.052250,0.913992,1.211422
1,rs377808,chr1:52724687:T:G,[EUR] FIGS,-0.051600,0.020222,0.010720,0.949709,0.912803,0.988106
2,rs377808,chr1:52724687:T:G,[AJ] GP2,-0.101593,0.082202,0.216497,0.903397,0.768966,1.061330
3,rs377808,chr1:52724687:T:G,[EUR] GP2,-0.060258,0.015734,0.000128,0.941522,0.912930,0.971009
4,rs377808,chr1:52724687:T:G,[EUR] IPDGC - Courage UK,-0.127412,0.075081,0.089698,0.880371,0.759899,1.019943
...,...,...,...,...,...,...,...,...,...
3318,rs8130097,chr21:45161847:C:A,[EUR] IPDGC - UK GWAS,0.081101,0.093659,0.386531,1.084480,0.902604,1.303005
3320,rs8130097,chr21:45161847:C:A,[EUR] Million Veteran Program,0.058583,0.039285,0.135900,1.060333,0.981753,1.145202
3321,rs8130097,chr21:45161847:C:A,[EUR] UKBiobank - Case/Control,0.147687,0.051833,0.004381,1.159150,1.047173,1.283101
3322,rs8130097,chr21:45161847:C:A,[EUR] UKBiobank - Proxy Cases,0.089411,0.051567,0.082937,1.093530,0.988408,1.209833


In [ ]:
cojo_ors_file2
cojo_ors_file2.rename(columns={"SNP(hg38)": "SNP"}, inplace=True)
cojo_ors_file2_rerrange = cojo_ors_file2[['rsID', 'SNP', 'Cohort', 'BETA', 'SE', 'P', 'OR', 'L95', 'U95']].copy()
cojo_ors_file2_rerrange

,rsID,SNP,Cohort,BETA,SE,P,OR,L95,U95
0,rs10569457,chr3:151990484:CGAAA:C,[ICE] deCODE,-0.057629,0.030890,0.062097,0.944000,0.888541,1.002920
1,rs10569457,chr3:151990484:CGAAA:C,[EUR] IPDGC - McGill,0.076878,0.109532,0.482758,1.079910,0.871268,1.338515
2,rs10569457,chr3:151990484:CGAAA:C,[EUR] IPDGC - NIA,-0.078313,0.081444,0.336272,0.924675,0.788248,1.084715
3,rs10569457,chr3:151990484:CGAAA:C,[AJ] FIGS,0.023228,0.083455,0.780758,1.023500,0.869060,1.205386
4,rs10569457,chr3:151990484:CGAAA:C,[EUR] IPDGC - Germany,-0.065666,0.106107,0.536008,0.936444,0.760609,1.152928
...,...,...,...,...,...,...,...,...,...
117,rs72712862,chr1:161000299:C:T,[EUR] UKBiobank - Case/Control,0.239174,0.099290,0.016003,1.270200,1.045573,1.543085
118,rs72712862,chr1:161000299:C:T,[EUR] IPDGC - UK GWAS,0.200022,0.196214,0.308008,1.221430,0.831473,1.794275
119,rs72712862,chr1:161000299:C:T,[EUR] FIGS,0.244900,0.124546,0.049260,1.277494,1.000789,1.630703
120,rs72712862,chr1:161000299:C:T,[ICE] deCODE,0.371564,0.107008,0.000516,1.450001,1.175658,1.788361


In [ ]:
cojo_ors_file2_rerrange = cojo_ors_file2_rerrange.iloc[1:]

merged_df = pd.concat([cojo_ors_file1, cojo_ors_file2_rerrange], ignore_index=True)
merged_df

,rsID,SNP,Cohort,BETA,SE,P,OR,L95,U95
0,rs377808,chr1:52724687:T:G,[AJ] FIGS,0.050931,0.071870,0.478523,1.052250,0.913992,1.211422
1,rs377808,chr1:52724687:T:G,[EUR] FIGS,-0.051600,0.020222,0.010720,0.949709,0.912803,0.988106
2,rs377808,chr1:52724687:T:G,[AJ] GP2,-0.101593,0.082202,0.216497,0.903397,0.768966,1.061330
3,rs377808,chr1:52724687:T:G,[EUR] GP2,-0.060258,0.015734,0.000128,0.941522,0.912930,0.971009
4,rs377808,chr1:52724687:T:G,[EUR] IPDGC - Courage UK,-0.127412,0.075081,0.089698,0.880371,0.759899,1.019943
...,...,...,...,...,...,...,...,...,...
3285,rs72712862,chr1:161000299:C:T,[EUR] UKBiobank - Case/Control,0.239174,0.099290,0.016003,1.270200,1.045573,1.543085
3286,rs72712862,chr1:161000299:C:T,[EUR] IPDGC - UK GWAS,0.200022,0.196214,0.308008,1.221430,0.831473,1.794275
3287,rs72712862,chr1:161000299:C:T,[EUR] FIGS,0.244900,0.124546,0.049260,1.277494,1.000789,1.630703
3288,rs72712862,chr1:161000299:C:T,[ICE] deCODE,0.371564,0.107008,0.000516,1.450001,1.175658,1.788361


In [ ]:
merged_df.to_csv(f'{WORK_DIR}/METAS/FINAL_SETS/FOREST_PLOTS/data/rsIDs_cojo_with_ORs_allCohorts_DEC2024.txt', index=False, sep="\t")

In [ ]:
## spot check a few 
! head -1 {WORK_DIR}/UKB/UKB_ALL/UKB_ALL_CC_FOR_PLINK_MAF
! grep "chr1:161000299:C:T" {WORK_DIR}/UKB/UKB_ALL/UKB_ALL_CC_FOR_PLINK_MAF

SNP	CHR	BP	A1	A2	BETA	SE	P	NMISS
chr1:161000299:C:T	1	161000299	T	C	0.23917436599731445	0.09928940236568451	0.0160034	15455.482915610626


In [ ]:
## spot check a few 
! head -1 {WORK_DIR}/MVP_CC_FOR_PLINK_MAF.txt
! grep "chr21:45161847:C:A" {WORK_DIR}/MVP_CC_FOR_PLINK_MAF.txt

SNP	CHR	BP	A1	A2	BETA	SE	P	NMISS
chr21:45161847:C:A	21	45161847	A	C	0.058582957834005356	0.039284877479076385	0.1359	34141.015226436444


In [ ]:
## spot check a few 
! head -1 {WORK_DIR}/MVP_CC_FOR_PLINK_MAF.txt
! grep "chr12:40340400" {WORK_DIR}/MVP_CC_FOR_PLINK_MAF.txt

SNP	CHR	BP	A1	A2	BETA	SE	P	NMISS
chr12:40340400:G:A	12	40340400	A	G	1.6378374099731445	0.24285557866096497	1.54e-11	34141.015226436444


In [ ]:
grouped = merged_df.groupby('rsID')

for rsID, group in grouped:
    filename = f"{WORK_DIR}/METAS/FINAL_SETS/FOREST_PLOTS/data/rsID_files_withMeta_cojo/{rsID}_split.txt"
    group.to_csv(filename, sep='\t', index=False)

print("Files have been successfully created.")

Files have been successfully created.


# Example - Forest Plot code
in R

In [ ]:
# Required Libraries
library(metafor)

# Function to clean filenames by removing problematic characters
clean_filename <- function(filename) {
  # Remove leading/trailing whitespace
  filename <- trimws(filename)
  # Replace spaces with underscores
  filename <- gsub(" ", "_", filename)
  # Remove single quotes
  filename <- gsub("'", "", filename)
  # Return cleaned filename
  return(filename)
}

# Read in rsID and nearest gene information
rsid_gene_info <- read.table("./METAS/FINAL_SETS/FOREST_PLOTS/data/rsID_and_nearest_gene.txt", header = TRUE, sep = "\t")

# Directories
input_directory <- "./METAS/FINAL_SETS/FOREST_PLOTS/data/rsID_files_withMeta_cojo/"
output_directory <- "./METAS/FINAL_SETS/FOREST_PLOTS/data/cojo_forest_plots/"
error_log_file <- "./METAS/FINAL_SETS/FOREST_PLOTS/data/error_log.txt"

dir.create(output_directory, showWarnings = FALSE)

# Initialize error log
write.table(data.frame(rsID = character(), Cohort = character(), Error = character()),
            file = error_log_file, sep = "\t", row.names = FALSE, col.names = TRUE, quote = FALSE)

# Files
files <- list.files(input_directory, pattern = ".txt$", full.names = TRUE)
cohort_order_path <- "./METAS/FINAL_SETS/FOREST_PLOTS/data/cohort_names.txt"
cohort_order <- read.table(cohort_order_path, header = TRUE, sep = "\t")

# Loop
for (file_path in files) {
  print(paste("Processing file:", file_path))
  
  # Read in data
  data <- read.table(file_path, header = TRUE, sep = "\t")
  
  # Check for empty or invalid data
  if (nrow(data) == 0 || all(is.na(data))) {
    print(paste("Skipping file due to invalid data:", file_path))
    next
  }
  
  # Sort by cohort order
  data <- data[match(cohort_order$Rename, data$Cohort), ]
  data <- na.omit(data)
  
  # Drop cohorts where BETA is zero and log the cohort in the error log
  zero_beta_cohorts <- data$Cohort[data$BETA == 0]
  if (length(zero_beta_cohorts) > 0) {
    for (cohort in zero_beta_cohorts) {
      rsID <- ifelse(nrow(data) > 0, as.character(data$rsID[1]), "Unknown")
      write.table(data.frame(rsID = rsID, Cohort = cohort, Error = "BETA is zero"),
                  file = error_log_file, sep = "\t", row.names = FALSE, col.names = FALSE, append = TRUE, quote = FALSE)
    }
    data <- data[data$BETA != 0, ]
  }
  
  # Check for missing values in key columns
  if (any(is.na(data$BETA) | is.na(data$SE))) {
    print(paste("Skipping file due to NA values in BETA or SE:", file_path))
    next
  }
  
  # Analysis
  labs_normal <- data$Cohort
  resFe_normal <- tryCatch({
    rma(yi = data$BETA, sei = data$SE, slab = labs_normal, method = "FE")
  }, error = function(e) {
    # Log the error and rsID to the error log file
    rsID <- ifelse(nrow(data) > 0, as.character(data$rsID[1]), "Unknown")
    write.table(data.frame(rsID = rsID, Cohort = "All", Error = e$message),
                file = error_log_file, sep = "\t", row.names = FALSE, col.names = FALSE, append = TRUE, quote = FALSE)
    print(paste("Error in rma for file:", file_path, " - ", e$message))
    return(NULL) # Skip to next iteration
  })
  
  # Skip plotting if `rma` failed
  if (is.null(resFe_normal)) {
    next
  }
  
  # Extract rsID and nearest gene for plot title
  rsid <- as.character(data$rsID[1])
  nearest_gene <- rsid_gene_info$Nearest_Gene[match(rsid, rsid_gene_info$rsID)]
  plot_title <- paste(rsid, "- Nearest Gene:", nearest_gene)
  
  # Plot
  output_file <- paste(output_directory, clean_filename(rsid), ".png", sep = "")
  png(file = output_file, width = 800, height = 700)
  
  Pvalue_normal <- formatC(resFe_normal$pval, digits = 4)
  options(na.action = "na.pass")
  par(mar = c(5, 4, 1, 1))
  
  forest(resFe_normal, annotate = TRUE, xlim = c(-2.25, 3.25), width = 3, cex.lab = 0.8, cex.axis = 1,
         atransf = exp, xlab = "Odds Ratio (95%CI)", slab = labs_normal, mlab = "[TOTAL] Meta-analysis",
         col = "red", border = "red", cex = 0.9, at = log(c(0.5, 1, 2, 3)))
  text(0, length(labs_normal) + 2, plot_title, cex = 1.5, font = 2)
  
  dev.off()
}